# 모듈 불러오기

#### 기본

In [ ]:
import numpy as np
import pandas as pd
import joblib

In [ ]:
# !conda install numpy 
# !conda install pandas
# !conda install scikit-learn
# !conda install scipy
# !conda install tensorflow
# !conda install matplotlib
# !conda install seaborn
# !pip install ydata-profiling

# !pip install xgboost
# !pip install lightgbm
# !pip install catboost
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !pip install hyperopt
# !pip install -U imbalanced-learn
# !pip install missingno
# !pip install shap

#### 전처리

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn import impute
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer

#### 리샘플링

In [ ]:
from imblearn.over_sampling import (
    RandomOverSampler, 
    ADASYN, 
    SMOTE
)
from imblearn.under_sampling import (
    RandomUnderSampler, 
    TomekLinks, 
    CondensedNearestNeighbour, 
    OneSidedSelection, 
    EditedNearestNeighbours, 
    NeighbourhoodCleaningRule
)

#### 분석

In [ ]:
from scipy.stats import skew, kurtosis
from scipy.stats import ttest_ind, f_oneway, pearsonr, chi2_contingency
from ydata_profiling import ProfileReport

#### 회귀

In [ ]:
from sklearn.linear_model import LinearRegression as RL
from sklearn.neighbors import KNeighborsRegressor as KNNR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR
from xgboost import XGBRegressor as XGBR
from lightgbm import LGBMRegressor as LGBMR
from catboost import CatBoostRegressor as CBR

from lightgbm import plot_importance as lgbm_plot_importance
from xgboost import plot_importance as xgb_plot_importance
from catboost import Pool

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

#### 분류

In [ ]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from xgboost import XGBClassifier as XGBC
from lightgbm import LGBMClassifier as LGBMC
from catboost import CatBoostClassifier as CBC

from sklearn.metrics import confusion_matrix as cmatrix
from sklearn.metrics import classification_report as creport
from sklearn.metrics import recall_score as recall
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

#### 교차검증

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import cv
import hyperopt

from sklearn.model_selection import (
    StratifiedKFold, # 분류
    KFold, # 회귀
    # GroupKFold, 
    # RepeatedKFold, 
    # StratifiedGroupKFold, 
    # RepeatedStratifiedKFold
)

#### 비즈니스 이해

In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay, partial_dependence

#### 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rc('font', family='Malgun Gothic')
sns.set(font="Malgun Gothic",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')  

import warnings
warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

tf.compat.v1.disable_v2_behavior() # shap 그래프 tf1 버전 지원 tf2 비활성화
import shap

# 피처 엔지니어링

#### FI (Feature Importance)
- LR, KNN, SVM 불가능
- DT, RF, 

In [ ]:
# 모델 1개만 플롯
tmp = pd.DataFrame({'feature_importance': model.feature_importances_, 'feature_names': list(x_train)}).sort_values('feature_importance', ascending=False)

plt.figure(figsize=(16, 6))
sns.barplot(x='feature_importance', y='feature_names', data = tmp)
plt.title('model_name')
plt.tight_layout()
plt.show()

In [ ]:
# 반복해서 플롯
for key in results:
    if key in ['RL', 'KNNR', 'SVR']:
        continue
    tmp = pd.DataFrame({'feature_importance': results[key].feature_importances_, 'feature_names': list(x_train)}).sort_values('feature_importance', ascending=False)[:20]
    plt.figure(figsize=(16, 6))
    sns.barplot(x='feature_importance', y='feature_names', data = tmp)
    plt.title(key)
    plt.tight_layout()
    plt.show()

In [ ]:
# feature importance 시각화 함수
def plot_feature_importance(importance, features, top_col = 'all'):
    feature_importance = np.array(importance)
    feature_names = np.array(features)

    data={'feature_names':feature_names,'feature_importance':feature_importance}
    tmp = pd.DataFrame(data)

    tmp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    tmp.reset_index(drop=True, inplace = True)

    if top_col == 'all' :
        features_df = tmp.copy()
    else :
        features_df = tmp.iloc[:top_col]

    plt.figure(figsize=(10,8))
    sns.barplot(x='feature_importance', y='feature_names', data = features_df)

    plt.xlabel('importance')
    plt.ylabel('feature names')
    plt.grid()

    return features_df

plot_feature_importance(model.feature_importances_, features=list(x_train), top_col=10)

#### PFI (Permutation Feature Importance)
- 모델과 상관없이 피처 중요도를 얻을 수 있다.
- 다중공선성(선형회귀, 로지스틱회귀에 중요)이 있으면 효과 없음

In [ ]:
pfi1 = permutation_importance(model, x_val_s, y_val, n_repeats=10, 
                              scoring = 'r2', random_state=20)

In [ ]:
sorted_idx = pfi.importances_mean.argsort()

plt.figure(figsize = (12,8))
plt.boxplot(pfi.importances[sorted_idx].T, vert=False, labels=x_train.columns[sorted_idx])
plt.axvline(0, color = 'r')
plt.grid()
plt.show()

#### PDP (Partial Dependence Plot)

In [ ]:
"""
def from_estimator(
    cls, # 학습한 모델
    estimator, # 살펴볼 분석단위, 데이터들
    X, # target
    features, # 분석할 변수
    categorical_features=None, 
    grid_resolution=100,
    n_jobs=None,
    kind='both', #  plotting both Partial dependence (PD) and individual conditional expectation (ICE)
    )
"""
PartialDependenceDisplay.from_estimator(model, x_train, target, kind="both", n_jobs=-1)
plt.grid()
plt.show()

# 데이터 저장

In [ ]:
folder_name = '1th'

x_train.to_csv(f'./{folder_name}/x_train.csv', index = False)
y_train.to_csv(f'./{folder_name}/y_train.csv', index = False)
x_test.to_csv(f'./{folder_name}/x_test.csv', index = False)
y_test.to_csv(f'./{folder_name}/y_test.csv', index = False)
y_test.to_csv(f'./{folder_name}/x_final.csv', index = False)